In [1]:
from glob import glob
import scanpy as sc
import pandas as pd
adata=sc.read("adata_cc.h5ad")

# prepare data for brie-count
allbam=pd.Series(glob("../*/star_mm10/*/*unique.bam"))
keep = []
for p,w in [tuple(n.split(":")) for n in adata.obs_names]:
    keep = keep+[allbam[allbam.str.contains(p) & allbam.str.contains(w)].values[0]]
    
pd.concat([pd.Series(keep),pd.Series(adata.obs_names)],axis=1).to_csv("BRIE2/bam_and_cellID.tsv",header=None,index=None,sep="\t")

# prepare data for brie-quant
cell_info=pd.DataFrame({"isTreated":(adata.obs.treatment=='5htp')*1,"isZO":(adata.obs.location == "ZO")*1}).reset_index()
cell_info.columns=["samID",'isTreated', 'isZO']
cell_info.to_csv("BRIE2/cell_info.tsv",index=None,sep="\t")

In [2]:
%%bash
find .. -name "*_unique.bam" -exec samtools index {} \;
source ~/miniconda3/etc/profile.d/conda.sh && conda activate BRIE 

brie-count -a BRIE2/SE.filtered.gff3 -S BRIE2/bam_and_cellID.tsv -o BRIE2/strict_filt -p 40

brie-quant -i BRIE2/strict_filt/brie_count.h5ad -o BRIE2/strict_filt/brie_quant_cell.h5ad -c BRIE2/cell_info.tsv --interceptMode gene --LRTindex=0

[['../SS2_20_127/star_mm10/A23/A23_unique.bam' 'SS2_20_127:A23']
 ['../SS2_20_127/star_mm10/A15/A15_unique.bam' 'SS2_20_127:A15']
 ['../SS2_20_127/star_mm10/A17/A17_unique.bam' 'SS2_20_127:A17']] ...
[BRIE2] loading gene annotations ... Done.
[BRIE2] loading reads for 5763 genes in 1361 sam files with 40 cores...
[BRIE2] [====================] 100.0% cells done in 390.7 sec.
[BRIE2] save matrix into h5ad ...
[BRIE2] 100.0% cells are matched with features
layers: ['isoform1', 'isoform2', 'ambiguous']
Filtered out 3014 genes with less than 50 total counts or 30 cells with unique counts or 10 unique counts or 0.0010 minor isoform frequency
AnnData object with n_obs × n_vars = 1361 × 2749
    obs: 'samID', 'samCOUNT'
    var: 'GeneID', 'GeneName', 'TranLens', 'TranIDs', 'chrom', 'ExonSS', 'n_counts', 'n_counts_uniq'
    varm: 'effLen', 'p_ambiguous'
    layers: 'ambiguous', 'isoform1', 'isoform2', 'poorQual'
[BRIE2] adding pseudo_count: 0.01
[BRIE2] model fit with 5000 steps in 0.59 min, l

2021-07-06 18:18:28.217947: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/lfaure/miniconda3/envs/BRIE/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'samCOUNT' as categorical
/home/lfaure/miniconda3/envs/BRIE/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'GeneName' as categorical
/home/lfaure/miniconda3/envs/BRIE/lib/python3.7/site-p